In [1]:
import pandas as pd
import re
from dateparser.search import search_dates

In [2]:
df = pd.read_csv('pattern.csv', encoding = 'cp1252')
df.head()

tables = []
for index, row in df.iterrows():
    tables.append(row['tableId'])

print(tables[1])
print(len(tables))

03bfc26a-c6d0-4761-b8f5-47acf2290d02
27


In [3]:
import mysql.connector
cnx = mysql.connector.connect(user='vibudh', password='vibudh3399',
                              host='os25.neb-one.gc.ca',
                              database='pcmr')
df = pd.read_sql('SELECT * FROM issues', con=cnx)
cnx.close()
df_issue = df[['tableId', 'rowIndex', 'status_bin', 'status_txt']]
print(len(df_issue))
df_issue.head()

5190


,tableId,rowIndex,status_bin,status_txt
0,0114daa6-c048-4081-b4cf-7a9d7461fa44,1,Unresolved,Monitor in 2018.
1,0114daa6-c048-4081-b4cf-7a9d7461fa44,2,Unresolved,Monitor in 2018 and implement weed control as ...
2,0114daa6-c048-4081-b4cf-7a9d7461fa44,3,Unresolved,Remove erosion log.
3,0114daa6-c048-4081-b4cf-7a9d7461fa44,4,Unresolved,Monitor in 2018 and implement weed control as ...
4,0114daa6-c048-4081-b4cf-7a9d7461fa44,5,Unresolved,Remove sediment fence.


In [4]:
#Checking for the rows which might have string length 0
for index, row in df_issue.iterrows():
    if row['tableId'] in tables:
        #print("---------------------", "---------------------", "---------------------", "---------------------", "---------------------")
        #print(row['status_txt'].replace("<s>", ""))
        if len(str(row['status_txt'])) <1:
            "hello"
#             print(row['status_txt'])
#             print(row['rowIndex'])
#             print(row['tableId'])

In [5]:
#Counting relevant rows. Rows which belong in the list of tables provided
count = 0
for index, row in df_issue.iterrows():
    try:
        if row['tableId'] in tables:
            count = count +1
    except:
        continue
print(count)

1946


In [6]:
def split_issues(status_txt):
    parsed_issues = []
    issue_str = ""
    
    date_string = ""
    date_count = 0
    
    status_txt_multiple = status_txt.split(".")
    
    i = 0
    new_issue_flag = 0
    new_issue_with_date_flag = 0 
    
    for line in status_txt_multiple:
#         print("line", line)
#         print("issue_str", issue_str)
        i = i+1
        new_issue_flag = 0
        new_issue_with_date_flag = 0
        if i == 1:
            issue_str = line
            date_string, date_count  = date_parser_date_count(line)
            #dates.append(dates_a[0])
            continue
            
        # usuallu a new issue starts with ". -", 
        # so afer splitting with "." if the first four characters contains - then we can say that it is a new issue
        if "-" in line.strip()[:2] or ("–" in line.strip()[:2]):
            new_issue_flag = 1
            parsed_issues.append((issue_str + ".").strip())
            issue_str = date_string + " " + line   
            
            
        # a new issue might also start like this "Issue. Sept 17, 2016 - 65-75% vegetation."
        # line 1: Issue.
        # line 2: Sept 17, 2016 - 65-75% vegetation.
        if len(line) > 25:
            date_s, date_c = date_parser_date_count(line)
            #print(date_string)
            if (date_c > 0) and (("–" in line.strip()[:21]) or ("-" in line.strip()[:21])):
                #dates.append(dates_a[0])
                new_issue_with_date_flag = 1
                parsed_issues.append((issue_str + ".").strip())
                issue_str = line
                date_string = date_s
                date_count = date_c
                
        
        if new_issue_flag == 0 and new_issue_with_date_flag == 0 and len(line) > 0:
            issue_str = issue_str + ". " + line.strip()
                
#         if flag == 1:
#             #print(issue_str)
#             parsed_issues.append(issue_str + ".")
#             issue_str = line
#             flag = 0
        
    if new_issue_with_date_flag ==1:
        parsed_issues.append(issue_str + ".")
        
    elif new_issue_flag ==1:
        parsed_issues.append((date_string + " " + issue_str + ".").strip())
        
    else:
        parsed_issues.append((issue_str + ".").strip())
    
    return parsed_issues
    

In [7]:
def date_parser_date_count(string):
    ''' Date Finding Function
        Input:  Takes any string as input 
        Output1: Returns the first date identified 
        Output2: Returns the number of dates identified in the string'''
    
    para = search_dates(string, 
                        settings={'STRICT_PARSING': True, 
                                  'DATE_ORDER': 'DMY'})
    date_string = ""
    count = 0
    try:
        for x in para:
            count = count+1
            if date_string == "":
                date_string = x[0]
    except:
        # No date identified or error in finding a date
        "Hello"
        
    return(date_string, count)

line = "July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract."
date_string, date_count  = date_parser_date_count(line)
print(date_string)
print(date_count)

July 13, 2016
1


In [8]:
text = "June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures. July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract. Good establishment of grasses, shrubs and wetland vegetation.  - Wetland functioning well, consistent with previous assessments. - Intermittent herbicide application conducted to control perennial sow thistle and scentless chamomile  - No erosion or subsidence issues. Sept 17, 2016 - 65-75% vegetation established compared to off ROW. - Intermittent herbicide application conducted to control perennial sow thistle."
print(text)
print("-----breakdown-----")

for row in split_issues(text):
    print(":   ", row)

June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures. July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract. Good establishment of grasses, shrubs and wetland vegetation.  - Wetland functioning well, consistent with previous assessments. - Intermittent herbicide application conducted to control perennial sow thistle and scentless chamomile  - No erosion or subsidence issues. Sept 17, 2016 - 65-75% vegetation established compared to off ROW. - Intermittent herbicide application conducted to control perennial sow thistle.
-----breakdown-----
:    June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures.
:    July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract. Good establishment of grasses, shrubs and wetland vegetation.
:    July 13, 2016   - Wetland functioning well, consistent with previous assessments.
:    July 13, 2016  - Intermittent herbicide a

In [9]:
text = "June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures. July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract. Good establishment of grasses, shrubs and wetland vegetation.  Wetland functioning well, consistent with previous assessments. - Intermittent herbicide application conducted to control perennial sow thistle and scentless chamomile.  - No erosion or subsidence issues. Sept 17, 2016 - 65-75% vegetation established compared to off ROW. - Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle.Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. "
print(text)
print("-----breakdown-----")

for row in split_issues(text):
    print(":   ", row)
    print(len(row))

June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures. July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract. Good establishment of grasses, shrubs and wetland vegetation.  Wetland functioning well, consistent with previous assessments. - Intermittent herbicide application conducted to control perennial sow thistle and scentless chamomile.  - No erosion or subsidence issues. Sept 17, 2016 - 65-75% vegetation established compared to off ROW. - Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle.Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermittent herbicide application conducted to control perennial sow thistle. Intermitten

In [10]:
text = "August 12, 2017 - KP 1+890 to 2+040 and KP 2+250 to 2+350 – Intermittent herbicide application conducted to control perennial sow thistle and tansy. Issue resolved. September 11, 2017 – Woody swamp on ROW from KP 1+920 to 1+960 and 2+200 to 2+360 appears to be at full functionality."
print(text)
print("-----breakdown-----")

split_issues(text)

August 12, 2017 - KP 1+890 to 2+040 and KP 2+250 to 2+350 – Intermittent herbicide application conducted to control perennial sow thistle and tansy. Issue resolved. September 11, 2017 – Woody swamp on ROW from KP 1+920 to 1+960 and 2+200 to 2+360 appears to be at full functionality.
-----breakdown-----


['August 12, 2017 - KP 1+890 to 2+040 and KP 2+250 to 2+350 – Intermittent herbicide application conducted to control perennial sow thistle and tansy. Issue resolved.',
 'September 11, 2017 – Woody swamp on ROW from KP 1+920 to 1+960 and 2+200 to 2+360 appears to be at full functionality.']

In [11]:
len(df_issue)

5190

In [12]:
tableId = []
rowIndex = []
status_bin =[]
status_txt = []
issues =[]
count = []

none_type = []
for index, row in df_issue.iterrows():
    try:
        if row['tableId'] in tables:
#             print("---------------------", "---------------------", "---------------------", "---------------------", "---------------------")
#             print(row["status_txt"])
        
            parsed_issues = split_issues(str(row["status_txt"]))
            counter = 0
#             print("parsed issues", parsed_issues)
#             print("HELLOOOOO")

            if len(parsed_issues) > 0:
                #print("--------------------- breakdown ---------------------")
                for issue in parsed_issues:
                    counter = counter + 1
                    #print("---", issue)
                    count.append(count)
                    tableId.append(row['tableId'])
                    rowIndex.append(row['rowIndex'])
                    status_bin.append(row['status_bin'])
                    status_txt.append(row['status_txt'])
                    issues.append(issue)
#                     if len(issue) > 260:
#                         print(index)
            #print(row['tableId'])
        else:
            continue
    except:
        none_type.append(index)
        
    

58
72
72
73
73
73
74
82
83
84
84
88
90
625
626
627
646
855
856
857
859
861
862
864
864
865
867
868
868
871
872
877
877
879
879
881
882
883
884
884
885
886
887
888
888
889
890
890
891
892
892
893
894
894
895
896
897
898
899
901
902
904
905
906
909
910
911
912
913
914
915
916
919
920
920
920
920
922
923
923
925
927
928
929
929
930
971
971
972
972
973
973
973
974
975
975
976
976
976
977
979
979
981
981
982
982
982
983
984
985
985
985
986
986
988
996
996
996
998
998
998
1006
1006
1034
1035
1072
1073
1074
1077
1078
1084
1084
1091
1092
1095
1095
1096
1096
1099
1102
1107
1108
1108
1109
1109
1110
1110
1112
1124
1124
1128
1138
1138
1138
1138
1138
1139
1139
1139
1139
1139
1145
1877
1878
1879
1880
1881
1882
1883
1883
1884
1885
1886
1887
1888
1889
1890
1891
1894
1895
1896
1897
1898
1899
1900
1901
1903
1904
1905
1907
1908
1910
1911
1912
1913
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1924
1926
1927
1928
1930
1931
1932
1933
1934
1936
1937
1939
1940
1941
1942
1943
1944
1945
1947
1948
1949
1950

In [ ]:
df_parsed_issues = pd.DataFrame({'tableId': tableId,
                          'rowIndex' : rowIndex,
                          'status_bin' : status_bin,
                          'status_txt' :  status_txt,
                        'count' : count,
                        'issues': issues})
df_parsed_issues.head(50)
#len(df_labels)
print(len(df_parsed_issues))

In [ ]:
print(len(none_type))
print(none_type)

In [ ]:
df_parsed_issues.to_csv("v2_parsed_issues.csv", index = False, encoding='utf-8-sig')

In [ ]:
df_parsed_issues.to_csv("v2_parsed_issues_tab.tsv", sep="\t")

In [ ]:
a = "hello I am going to write a very long string. A string which has more than 255 characters to see if a very large string object behaves under string function. hello I am going to write a very long string. A string which has more than 255 characters to see if a very large string object behaves under string function."
print(len(a))
print(len(str(a)))